In [1]:
%matplotlib inline
import numpy as np
import matplotlib.tri as Tri
import matplotlib.pyplot as plt
#from mpl_toolkits.basemap import Basemap
from netCDF4 import Dataset as NetCDFFile 
import netCDF4
from netCDF4 import date2index
import datetime as dt
import pandas as pd
from StringIO import StringIO
import xarray as xr

/Users/bgh2-ra/anaconda2/lib/python2.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/bgh2-ra/anaconda2/lib/python2.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
import re

def extractURLsatellite(fileURL, satName):
    """
    Function to extract the URLs for a specific satellite from the IMOS URLs list
    
    Parameters:
    -----------
    
    - fileURL : IMOS URLs list as a txt file
    - satName : name of the satellite such as JASON-2 JASON-3 
    
    Ouputs:
    ------
    
    - getFiles : list of URLs for the desired satellite
    
    """
    
    getFiles = []
    
    with open(fileURL) as f:
        for line in f:
            if re.search(r"%s"%satName, line):
                changeURL = re.sub('http://data.aodn.org.au', 'http://thredds.aodn.org.au/thredds/dodsC', line)
                getFiles.append(changeURL)
                
    return getFiles

In [3]:
jason2URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'JASON-2')

jason3URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'JASON-3')

saralURL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'SARAL')

sentinel3aURL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'SENTINEL-3A')

cryosat2URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'CRYOSAT-2')

enviURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'ENVISAT')

geosatURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'GEOSAT')

ersURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'ERS-2')

gfoURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'GFO')

topURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'TOPEX')

In [4]:
allURL = []
allURL.append(jason2URL)
allURL.append(jason3URL)
allURL.append(saralURL)
allURL.append(sentinel3aURL)
allURL.append(cryosat2URL)
allURL.append(enviURL)
allURL.append(geosatURL)
allURL.append(ersURL)
allURL.append(gfoURL)
allURL.append(topURL)

In [5]:
boxLat = [] #latitude
boxLon = [] #longitude
boxWh = [] #wave height
boxT = [] #time
boxQ = [] #quality control flag
boxB = [] #backscatter
boxU = [] #wind Speed

In [6]:
#Cape Melville coordinates
latmin = -14.0
latmax = -13.0  
lonmin = 145.0
lonmax = 146.0

if latmin>latmax:
    print('Error wrong definition of min and max lat!!!')

if lonmin>lonmax:
    print('Error wrong definition of min and max lon!!!')

In [7]:
# 
start_date = dt.datetime(1985,1,1)
end_date = dt.datetime(2019,2,21)

#start_date = dt.datetime(2010,1,1)
#end_date = dt.datetime(2010,12,31)

In [8]:
for u in range (len(allURL)):
    urlON = allURL[u]
    for k in range (len(urlON)):
        ncs = NetCDFFile(urlON[k])
        lats = ncs.variables['LATITUDE'][:]
        lons = ncs.variables['LONGITUDE'][:]
        ws = ncs.variables['WSPD_CAL'][:]
       
        if u == 2:
            wh = ncs.variables['SWH_KA_CAL'][:]
            qc = ncs.variables['SWH_KA_quality_control'][:]
            back = ncs.variables['SIG0_KA'][:]
            
        else:
            wh = ncs.variables['SWH_KU_CAL'][:]
            qc = ncs.variables['SWH_KU_quality_control'][:]
            back = ncs.variables['SIG0_KU'][:]

            
        # Get desired time interval  
        time_var = ncs.variables['TIME']
        tt = ncs.variables['TIME'][:]
        
        timing = netCDF4.num2date(tt,time_var.units)
        
        #data in correct time, quality flag and lat/lon
        for k in range(len(timing)):
            if timing[k] >= start_date and timing[k] <= end_date:
                if lats[k]>latmin and lats[k]<latmax and lons[k]>lonmin and lons[k]<lonmax:
                    boxLat.append(lats[k])
                    boxLon.append(lons[k])
                    boxWh.append(wh[k])
                    boxT.append(tt[k])
                    boxQ.append(qc[k])
                    boxB.append(back[k])
                    boxU.append(ws[k])

In [9]:
len(boxT)

36666

In [10]:
for k in range(len(boxLat)):
    if k == 0:
        lat = boxLat[k]
        lon = boxLon[k]
        wh = boxWh[k]
        tt = boxT[k]
        qc = boxQ[k]
        back = boxB[k]
        ws = boxU[k]
    else:
        lat = np.append(lat,boxLat[k])
        lon = np.append(lon,boxLon[k])
        wh = np.append(wh,boxWh[k])
        tt = np.append(tt,boxT[k])
        qc = np.append(qc,boxQ[k])
        back = np.append(back,boxB[k])
        ws = np.append(ws, BoxU[k])


NameError: name 'BoxU' is not defined

In [ ]:
print len(wh)
print len(lat)
print len(lon)

In [ ]:
ids = np.where(np.logical_and(wh>0,qc==1))
whpos = wh[ids]
qc_ = qc[ids]
lon_ = lon[ids]
tt_ = tt[ids]
lat_ = lat[ids]
back_ = back[ids]

print len(whpos), 'whpos'
print len(qc_),'qc_'
print len(lon_),'lon_'
print len(tt_),'tt_'
print len(lat_),'lat_'

In [ ]:
print 'mean whpos', np.mean(whpos)
print 'max whpos',max(whpos)
print 'min whpos', min (whpos)

print 'min lon',min(lon)
print 'min lon',min(lon)
print 'max lon',max(lon)

print 'min lat', min(lat)
print 'max lat',max(lat)

print min(netCDF4.num2date(tt_,time_var.units))
print max (netCDF4.num2date(tt_,time_var.units))

In [ ]:
len(whpos)
print min(netCDF4.num2date(tt_,time_var.units))
print max(netCDF4.num2date(tt_,time_var.units))

In [ ]:
for k in range(len(whpos)):
    #if np.where(np.logical_and(wh>0,qc==1)):
    print netCDF4.num2date(whpos[k],time_var.units)

In [ ]:
for k in range(len(whpos)):
    #if np.where(np.logical_and(wh>0,qc==1)):
    print netCDF4.num2date(tt_[k],time_var.units)

In [ ]:
sorted1 = np.sort(tt_)
for k in range(len(sorted1)):
    print netCDF4.num2date(sorted1[k],time_var.units)

In [ ]:
#find time of large wave height

for k in range(len(sorted1)):
    if whpos[k]>4.5:
        print netCDF4.num2date(sorted1[k],time_var.units)
         

# Practice from xr sheet

In [ ]:
pd.to_datetime(['2000-01-01', '2000-01-03'])

In [ ]:
pd.date_range('2000-01-01', periods=365)

In [ ]:
import datetime
xr.Dataset({'time': datetime.datetime(2000, 1, 1)})

In [ ]:
attrs = {'units': 'hours since 2000-01-01'}
ds = xr.Dataset({'time': ('time', [0, 1, 2, 3], attrs)})
xr.decode_cf(ds)

In [ ]:
time = pd.date_range('2000-01-01', freq='H', periods=365 * 24)
ds = xr.Dataset({'foo': ('time', np.arange(365 * 24)), 'time': time})
ds.sel(time='2000-01')

In [ ]:
ds.sel(time=slice('2000-06-01', '2000-06-10'))

In [ ]:
ds.sel(time=datetime.time(2))

In [ ]:
time = pd.date_range('2000-01-01', freq='6H', periods=365 * 4)

In [ ]:
ds = xr.Dataset({'foo': ('time', np.arange(365 * 4)), 'time': time})
ds.time.dt.hour

In [ ]:
ds.time.dt.dayofweek

In [ ]:
ds['time.month']


In [ ]:
ds['time.dayofyear']

In [ ]:
ds['time.season']

In [ ]:
ds.groupby('time.hour').mean()

In [ ]:
xr.Dataset({'tt_':datetime.datetime(2008-7-5 23:33:59.296878
)})

In [ ]:
newfile = xr.open_dataset('allURL')

wave age. g = 9.80665 m/s2

In [ ]:
w_a = 3.25*((whpos^2*g^2)/U^4)^0.31

GA-2

In [ ]:
tz = (((w_a-(5.78))/(w_a+(U/(whpos*((U/whpos)+whpos1)))))+(whpos+(5.7)))